In [ ]:
import pickle
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset
from torch_geometric.data import Data
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [21]:
import torch
import numpy as np
from torch_geometric.data import Data

def construct_graph(data_sample, threshold=0.8):
    """
    Constructs a graph for a single data sample from IEMOCAP.
    
    Args:
        data_sample: A dictionary containing 'audio_embed' and 'label'.
        threshold: Similarity threshold for connecting nodes based on embeddings.
    
    Returns:
        A PyG Data object representing the graph.
    """
    audio_embeds = data_sample['audio_embed']  
    labels = data_sample['label']              
    num_nodes = len(audio_embeds)
    
    x = torch.tensor(audio_embeds, dtype=torch.float)
    
    edge_index = []
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                sim = np.dot(audio_embeds[i], audio_embeds[j]) / (
                    np.linalg.norm(audio_embeds[i]) * np.linalg.norm(audio_embeds[j])
                )
                if sim >= threshold:
                    edge_index.append([i, j])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous() 
    
    y = torch.tensor(labels, dtype=torch.long)
    
    graph = Data(x=x, edge_index=edge_index, y=y)
    return graph

sample_data = {
    'audio_embed': np.random.rand(5, 128),  
    'label': [0, 1, 0, 2, 1],             
}

graph = construct_graph(sample_data)

print("Graph:")
print("Node Features (x):", graph.x.shape)  # (num_nodes, embed_dim)
print("Edge Index (edge_index):", graph.edge_index.shape)  # (2, num_edges)
print("Labels (y):", graph.y.shape)  # (num_nodes,)


Graph:
Node Features (x): torch.Size([5, 128])
Edge Index (edge_index): torch.Size([0])
Labels (y): torch.Size([5])


In [ ]:
BATCH_SIZE = 128
train_metadata = "C:/Users/admin/Documents/Speech-Emotion_Recognition-2/features/IEMOCAP_ECAPA_train.pkl"
val_metadata = "C:/Users/admin/Documents/Speech-Emotion_Recognition-2/features/IEMOCAP_ECAPA_train.pkl"
# train_dataset = Customized_Dataset(train_metadata)
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_dataset = Customized_Dataset(val_metadata)
# val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import numpy as np
import torch

def create_adj_matrix(num_nodes):
    """
    Create an adjacency matrix for a fully connected graph of num_nodes.
    """
    # Fully connected graph adjacency matrix
    adj_matrix = np.ones((num_nodes, num_nodes)) - np.eye(num_nodes)  # Remove self-loops
    return adj_matrix

def create_node_features(num_nodes, feature_dim):
    """
    Create random node features for a graph.
    """
    # Randomly initialize node features
    return np.random.rand(num_nodes, feature_dim)

# Example usage for a dataset
num_samples = 10  # Number of graphs in the dataset
num_nodes = 5     # Nodes per graph
feature_dim = 16  # Feature dimension per node

adj_matrices = []
node_features = []

for _ in range(num_samples):
    # Create adjacency matrix
    adj_matrices.append(create_adj_matrix(num_nodes))

    # Create node features
    node_features.append(create_node_features(num_nodes, feature_dim))

# Convert to PyTorch tensors if needed
adj_matrices = [torch.tensor(adj, dtype=torch.float) for adj in adj_matrices]
node_features = [torch.tensor(features, dtype=torch.float) for features in node_features]

# Verify shapes
print("Adjacency Matrix Shape:", adj_matrices[0].shape)  
print("Node Features Shape:", node_features[0].shape)    


Adjacency Matrix Shape: torch.Size([5, 5])
Node Features Shape: torch.Size([5, 16])
